## Problem:

We've got a building with 100 floors. One specific floor is the highest point you can drop an egg from without it breaking. Drop it from any floor above, and... *crack*! But if dropped from that magical floor or any floor below it, the egg remains intact and can be dropped again.

With just two eggs in hand, who can figure out the strategy to determine the highest safe floor with the fewest drops?

## Methodology

We have two eggs, but once the first one cracks, the only strategy that ensures we find the magic floor without cracking the second egg is to start at the floor just above the highest safe floor and move up one by one until we crack the egg. Therefore the following code just focusses on optimising the first guesses.

In [107]:
from typing import List
import random
random.seed(42)

In [133]:
def print_metrics(guesses: List[int]):
    '''
    Simulate guess strategy for all 100 magic floor scenarios and print summary metrics
    '''
    total_guesses = []    
    for magic_floor in range(1, 101):
        cur_guesses = []
        prev_guess = 0
        for guess in guesses:
            cur_guesses.append(guess)
            if guess <= magic_floor:
                prev_guess = guess
            else:
                num_left_to_check = magic_floor - prev_guess
                
                if guess != magic_floor + 1:
                    # We also need to the floor above the magic_floor
                    num_left_to_check += 1
                
                total_guesses.append(len(cur_guesses) + num_left_to_check)
                break
                
                
    print(f"Guesses: {guesses}, avg: {sum(total_guesses)/len(total_guesses):0.2f}, max: {max(total_guesses)}")

# Maths Solution

The mathematically optimal solution to reduce the max required drops is to use the triangle function x, x + (x-1), x + (x-1) + (x-2) ....  giving x=14

In [135]:
triangle_strategy = [sum([14 - i for i in range(j)]) for j in range(1, 15)]
print_metrics(triangle_strategy)

Guesses: [14, 27, 39, 50, 60, 69, 77, 84, 90, 95, 99, 102, 104, 105], avg: 10.48, max: 14


# Reinforcement Learning Approach

Q Learning: https://en.wikipedia.org/wiki/Q-learning

In [156]:
lr = 0.9 # learning rate (decays over time)
discount = 0.99
p_explore = 0.99 # probability of exploring vs exploiting
training_loops = 1_500_000

state_dict = {i: [0 for j in range(i+1, 101)] for i in range(100)}

for epoch in range(training_loops):
    
    if epoch != 0 and epoch % 100_000 == 0:
        # Every 100,000 epochs print metrics for current state
        guesses = []
        guess = 0
        while guess < 100:
            best_idx, _ = max(enumerate(state_dict[guess]), key=lambda x: x[1])
            guess = guess + best_idx + 1
            guesses.append(guess)
            
        print(f"Epoch {epoch}:", end=" ") 
        print_metrics(guesses)

        # decay explore and learning rate params
        p_explore *= 0.5
        lr *= 0.7
        
        
    magic_floor = random.randint(1, 100)
    prev_guess = 0
    done = False
    
    while not done:
        if random.random() < p_explore:
            next_guess = random.randint(prev_guess+1, 100)
        else:
            best_idx, _ = max(enumerate(state_dict[prev_guess]), key=lambda x: x[1])
            next_guess = prev_guess + best_idx + 1

        done = next_guess > magic_floor or next_guess == 100

        learned_val = 100 - next_guess + magic_floor if done else discount * max(state_dict[next_guess])

        state_dict[prev_guess][next_guess-prev_guess-1] = (1-lr) * state_dict[prev_guess][next_guess-prev_guess-1] + lr * learned_val

        prev_guess = next_guess

Epoch 100000: Guesses: [45, 58, 97, 98, 99, 100], avg: 21.25, max: 45
Epoch 200000: Guesses: [11, 19, 65, 73, 88, 94, 98, 100], avg: 17.32, max: 48
Epoch 300000: Guesses: [8, 20, 34, 45, 52, 61, 71, 78, 85, 89, 93, 96, 99, 100], avg: 10.66, max: 16
Epoch 400000: Guesses: [11, 19, 31, 39, 46, 54, 62, 68, 74, 85, 91, 94, 96, 99, 100], avg: 10.86, max: 20
Epoch 500000: Guesses: [13, 26, 37, 46, 54, 63, 71, 77, 81, 86, 91, 94, 98, 100], avg: 10.47, max: 16
Epoch 600000: Guesses: [12, 20, 32, 43, 52, 60, 68, 80, 85, 90, 94, 97, 99, 100], avg: 10.66, max: 19
Epoch 700000: Guesses: [11, 21, 33, 38, 50, 59, 67, 73, 80, 85, 90, 94, 97, 99, 100], avg: 10.69, max: 16
Epoch 800000: Guesses: [11, 26, 37, 47, 54, 61, 67, 73, 80, 85, 91, 95, 97, 99, 100], avg: 10.57, max: 16
Epoch 900000: Guesses: [11, 22, 31, 41, 50, 58, 67, 74, 80, 85, 90, 94, 97, 99, 100], avg: 10.52, max: 15
Epoch 1000000: Guesses: [11, 22, 31, 42, 50, 58, 67, 74, 80, 85, 90, 94, 97, 99, 100], avg: 10.53, max: 15
Epoch 1100000: G